In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy, qiskit
import circuit_cutter
import mlrecon_methods as ml
import json

import numpy as np
import qiskit
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

import qiskit.providers.aer.noise as noise
from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, H_distance_dict, total_counts
from utils.vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation
from mlrecon_methods import run_circuits, collect_fragment_circuits

from utils.utils import filter_results, dict_to_list, H_distance, total_counts, norm_dict
from utils.vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation
from mlrecon_methods import run_circuits, collect_fragment_circuits, organize_tomography_data_from_list

In [3]:
with open('BeH2_auckland_nocut_orign_rerun.json') as f:
    all_list = json.load(f)

In [4]:
len(all_list)

238

In [7]:
unmiti_list = all_list[-34:]
len(unmiti_list)

34

In [8]:
nocut_list = all_list[0:-34]
len(nocut_list)

204

In [9]:
with open('BeH2_auckland_check0.json') as f:
    miti_q0_list = json.load(f)
with open('BeH2_auckland_check1.json') as f:
    miti_q1_list = json.load(f)
with open('BeH2_auckland_check2.json') as f:
    miti_q2_list = json.load(f)
with open('BeH2_auckland_check3.json') as f:
    miti_q3_list = json.load(f)
with open('BeH2_auckland_check4.json') as f:
    miti_q4_list = json.load(f)
with open('BeH2_auckland_check5.json') as f:
    miti_q5_list = json.load(f)

In [10]:
len(nocut_list)

204

In [11]:
204/6

34.0

In [12]:
per_miti_results = []
for i in range(0, 6):
    temp_list = []
    for j in range(0, 34):
        filter_per_miti = norm_dict(filter_results(nocut_list[i * 34 + j], [0]))
        temp_list.append(filter_per_miti)
    per_miti_results.append(temp_list)

In [13]:
per_miti_results[0]

[{'000000': 0.01845018450184502,
  '000001': 0.013807880014284013,
  '010000': 0.019402452089037018,
  '010001': 0.013688846565885014,
  '010010': 0.01809308415664802,
  '010011': 0.013331746220688013,
  '010100': 0.02142602071182002,
  '010101': 0.015712415188668016,
  '010110': 0.017735983811451018,
  '010111': 0.012617545530294013,
  '011000': 0.016783716224259018,
  '011001': 0.011903344839900012,
  '011010': 0.017974050708249017,
  '011011': 0.013331746220688013,
  '011100': 0.01833115105344602,
  '011101': 0.016069515533865018,
  '011110': 0.017259850017855016,
  '011111': 0.013569813117486014,
  '000010': 0.01809308415664802,
  '100000': 0.018569217950244018,
  '100001': 0.01071301035591001,
  '100010': 0.02047375312462802,
  '100011': 0.015593381740269016,
  '100100': 0.01916438519223902,
  '100101': 0.013331746220688013,
  '100110': 0.02071182002142602,
  '100111': 0.015474348291870015,
  '101000': 0.015474348291870015,
  '101001': 0.012141411736698013,
  '101010': 0.016307582

Untimigated distribution

In [14]:
qubits = 6
# list of all possible measurement outcomes (bitstrings)
all_bits = [ "".join(bits) for bits in itertools.product(["0","1"], repeat = qubits) ]

In [15]:
def load_probs(miti_dist_list, bit_str):
    output_list = []
    for dist in miti_dist_list:
        try:
            miti_count = dist[bit_str]
        except:
            miti_count = 0
        output_list.append(miti_count)
    return output_list

In [16]:
def bit_weight(dist, index):
    #bitwise distribution
    weight_0 = 0
    weight_1 = 0
    for key in dist.keys():
        if key[len(key) - 1 - index] == '0':
            weight_0 += dist[key]
        elif key[len(key) - 1 - index] == '1':
            weight_1 += dist[key]
        else:
            print("Incorrect key value")
    return weight_0, weight_1

In [17]:
def update_dist(unmiti_dist, miti_dist, index):
    Ppost = {}
    w0, w1 = bit_weight(miti_dist, index)
    u_w0, u_w1 = bit_weight(unmiti_dist, index)
    if w0 == 0:
        w0 = 0.0000000000001
        w1 = 0.9999999999999
    if w1 == 0:
        w1 = 0.0000000000001
        w0 = 0.9999999999999
    if u_w0 == 0:
        u_w0 = 0.0000000000001
        u_w1 = 0.9999999999999
    if u_w1 == 0:
        u_w1 = 0.0000000000001
        u_w0 = 0.9999999999999
        
    for key in unmiti_dist.keys():
        if key[len(key) - 1 - index] == '0':
            Ppost[key] = unmiti_dist[key] / u_w0 * (w0)# / w1)
            #print(w0, w1, w0/w1, Ppost[key])
        elif key[len(key) - 1 - index] == '1':
            Ppost[key] = unmiti_dist[key] / u_w1 * (w1)# / w0)
            #print(w0, w1, w1/w0, Ppost[key])
        else:
            print("Incorrect key value")
    return Ppost

In [18]:
def combine_dist(orign_dist, dist_list):
    output_dist = {}
    for key in orign_dist:
        value = orign_dist[key]
        for dist in dist_list:
            value += dist[key]
        output_dist[key] = value
    return output_dist  

In [19]:
def bayesian_reconstruct(unmiti_dist, miti_dist_list, threshold = 0.0001):
    temp_dist = unmiti_dist.copy()
    h_dist = 1
    while h_dist > threshold:
        temp_dist_start = temp_dist.copy()
        ppost = [0] * len(miti_dist_list)
        for i in range(0, len(miti_dist_list)):
            ppost[i] = update_dist(temp_dist, miti_dist_list[i][0], miti_dist_list[i][1])
        temp_dist = combine_dist(temp_dist, ppost)
        temp_dist = norm_dict(temp_dist)
        h_dist = H_distance_dict(temp_dist, temp_dist_start)
        print("H-dist:", h_dist)
    return temp_dist

In [20]:
def norm_dict(dictionary):
    total = total_counts(dictionary)
    norm_dist = {}
    for i in dictionary.keys():
        norm_dist[i] = dictionary[i]/total
    return norm_dist

In [21]:
full_dist = []
for idx in range(0, len(per_miti_results[0])):
    miti_dist_index = [ [per_miti_results[0][idx],0],[per_miti_results[1][idx],1], [per_miti_results[2][idx],2], [per_miti_results[3][idx],3], [per_miti_results[4][idx],4], [per_miti_results[5][idx],5]]
    post_dist = bayesian_reconstruct(unmiti_list[idx], miti_dist_index)
    full_dist.append(post_dist)

H-dist: 70.00357133753646
H-dist: 0.01393861443516495
H-dist: 0.011865360966849826
H-dist: 0.01011925939613788
H-dist: 0.00864241945555635
H-dist: 0.007389360728611043
H-dist: 0.006323635418837053
H-dist: 0.005415582841444037
H-dist: 0.004640779518661337
H-dist: 0.003978932521232577
H-dist: 0.0034130647854299094
H-dist: 0.0029288988965399648
H-dist: 0.0025143798936463593
H-dist: 0.0021592982862456826
H-dist: 0.0018549872743917086
H-dist: 0.0015940762812884457
H-dist: 0.00137028816513381
H-dist: 0.0011782709567808585
H-dist: 0.0010134573245592698
H-dist: 0.0008719465984483207
H-dist: 0.0007504053425769399
H-dist: 0.000645983305567486
H-dist: 0.000556242203548649
H-dist: 0.0004790952663215375
H-dist: 0.0004127558463944437
H-dist: 0.0003556936823779466
H-dist: 0.00030659764234119096
H-dist: 0.0002643439629617795
H-dist: 0.0002279691564756597
H-dist: 0.00019664688670959946
H-dist: 0.00016966822317826194
H-dist: 0.0001464247724146333
H-dist: 0.00012639426151250165
H-dist: 0.0001091282127871

H-dist: 0.00018203263661934244
H-dist: 0.0001577010943133036
H-dist: 0.00013663777731230713
H-dist: 0.00011840109752483922
H-dist: 0.00010260955846269888
H-dist: 8.893350666426437e-05
H-dist: 70.0035713381083
H-dist: 0.036463087111371445
H-dist: 0.025502981429560395
H-dist: 0.01960727504594486
H-dist: 0.01569226622072455
H-dist: 0.012834591206895963
H-dist: 0.01063747757696049
H-dist: 0.008893912170719125
H-dist: 0.0074814490643756795
H-dist: 0.006320986776943024
H-dist: 0.005358011221967508
H-dist: 0.004553085949608009
H-dist: 0.003876611435675085
H-dist: 0.003305736144065299
H-dist: 0.002822431063721099
H-dist: 0.00241223098541865
H-dist: 0.002063377250580036
H-dist: 0.0017662129023054312
H-dist: 0.001512742705094458
H-dist: 0.001296304606809361
H-dist: 0.0011113188827945916
H-dist: 0.0009530929449395799
H-dist: 0.0008176670339661136
H-dist: 0.0007016906012913889
H-dist: 0.0006023221745180699
H-dist: 0.0005171474951547391
H-dist: 0.0004441120802389057
H-dist: 0.0003814653118380297
H-

In [22]:
pauli_list = read_from_file('BeH2_hamiltonian.txt')
pauli_commute = find_commute_groups(pauli_list)

In [23]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(full_dist[i], shots = 1, Pauli = Pauli_tuple[0])  

In [24]:
#results without cut
final_expect_val

(-3.3611878651503235+0j)

In [25]:
#with cutting
full_dist = []
for idx in range(0, len(per_miti_results[0])):
    miti_dist_index = [ [miti_q0_list[idx],0],[miti_q1_list[idx],1], [miti_q2_list[idx],2], [miti_q3_list[idx],3], [miti_q4_list[idx],4], [miti_q5_list[idx],5]]
    post_dist = bayesian_reconstruct(unmiti_list[idx], miti_dist_index)
    full_dist.append(post_dist)

H-dist: 70.00357133755685
H-dist: 0.015871136139910567
H-dist: 0.01333155985700341
H-dist: 0.011217285605807691
H-dist: 0.00944970759355526
H-dist: 0.007967596546195974
H-dist: 0.00672222791304993
H-dist: 0.005674203894350625
H-dist: 0.004791289637509355
H-dist: 0.004046881364600509
H-dist: 0.0034188834441696374
H-dist: 0.002888859989875847
H-dist: 0.002441377405563951
H-dist: 0.00206348428261321
H-dist: 0.001744293189900968
H-dist: 0.0014746401338133791
H-dist: 0.0012468046051696878
H-dist: 0.001054277787885886
H-dist: 0.0008915696304269244
H-dist: 0.0007540476425122214
H-dist: 0.0006378018195815053
H-dist: 0.0005395312273999993
H-dist: 0.0004564486308808864
H-dist: 0.0003862002087748107
H-dist: 0.0003267979139549363
H-dist: 0.00027656245401399805
H-dist: 0.00023407520366704337
H-dist: 0.00019813763653809936
H-dist: 0.0001677370919713033
H-dist: 0.00014201788197886844
H-dist: 0.00012025690151629744
H-dist: 0.00010184303757018719
H-dist: 8.625978351106582e-05
H-dist: 70.00357133792839


H-dist: 0.0057548080377312115
H-dist: 0.005238005743492275
H-dist: 0.004775674677302763
H-dist: 0.0043608985377479
H-dist: 0.003987790959054781
H-dist: 0.003651321597955626
H-dist: 0.003347175884203392
H-dist: 0.003071640582796647
H-dist: 0.002821509564387344
H-dist: 0.002594005680779065
H-dist: 0.0023867156628539173
H-dist: 0.0021975356679038473
H-dist: 0.0020246256095578386
H-dist: 0.0018663707746820214
H-dist: 0.001721349511508516
H-dist: 0.0015883059899214438
H-dist: 0.0014661272064995542
H-dist: 0.0013538235455378134
H-dist: 0.0012505123208429185
H-dist: 0.0011554038171158213
H-dist: 0.0010677894281009462
H-dist: 0.0009870315542704474
H-dist: 0.0009125549778191624
H-dist: 0.0008438394789081135
H-dist: 0.0007804134958206769
H-dist: 0.0007218486641541875
H-dist: 0.0006677550973412306
H-dist: 0.0006177772935097438
H-dist: 0.0005715905726537489
H-dist: 0.0005288979638973109
H-dist: 0.0004894274758021427
H-dist: 0.0004529296936258356
H-dist: 0.00041917565654817036
H-dist: 0.00038795497

H-dist: 0.006928434234515719
H-dist: 0.006406624782355332
H-dist: 0.005926052694556603
H-dist: 0.005482900411303139
H-dist: 0.005073848587280833
H-dist: 0.004695980915362377
H-dist: 0.004346711154759728
H-dist: 0.004023726927393671
H-dist: 0.0037249461568421263
H-dist: 0.003448483032778669
H-dist: 0.0031926211547324856
H-dist: 0.0029557920948668328
H-dist: 0.002736558062469518
H-dist: 0.0025335976863504086
H-dist: 0.0023456941814472554
H-dist: 0.002171725352867125
H-dist: 0.0020106550298735204
H-dist: 0.0018615256258300782
H-dist: 0.0017234515968384645
H-dist: 0.0015956136285672532
H-dist: 0.0014772534226847375
H-dist: 0.001367668985222689
H-dist: 0.00126621034198237
H-dist: 0.0011722756228867096
H-dist: 0.001085307469572332
H-dist: 0.0010047897296776003
H-dist: 0.0009302444080861285
H-dist: 0.0008612288504675263
H-dist: 0.000797333138286644
H-dist: 0.0007381776773715276
H-dist: 0.0006834109643875077
H-dist: 0.0006327075173413169
H-dist: 0.000585765957665193
H-dist: 0.00054230723260093

In [26]:
final_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        final_expect_val += coeff * evaluation(full_dist[i], shots = 1, Pauli = Pauli_tuple[0])  

In [27]:
final_expect_val

(-3.4288596525989172+0j)

In [28]:
#no bayesian recombine:
miti_dist_list = [miti_q0_list, miti_q1_list, miti_q2_list, miti_q3_list, miti_q4_list, miti_q5_list]
total = 0
for idx in range(0, 6):
    final_expect_val = 0
    for i in range(0, len(pauli_commute)):
        group = pauli_commute[i]
        for Pauli_tuple in group:
            coeff = Pauli_tuple[1]
            final_expect_val += coeff * evaluation(miti_dist_list[idx][i], shots = 1, Pauli = Pauli_tuple[0])  
    total += final_expect_val
    print("expectation value for mitigating qubit{}:{}".format(idx, final_expect_val) )

expectation value for mitigating qubit0:(-3.386702477271455+0j)
expectation value for mitigating qubit1:(-3.376942626397123+0j)
expectation value for mitigating qubit2:(-3.3938414179594116+0j)
expectation value for mitigating qubit3:(-3.3581690081275855+0j)
expectation value for mitigating qubit4:(-3.34132956096899+0j)
expectation value for mitigating qubit5:(-3.3103598298269437+0j)


In [29]:
total/6

(-3.361224153425251+0j)

In [30]:
#no cutting
total = 0
for idx in range(0, 6):
    final_expect_val = 0
    for i in range(0, len(pauli_commute)):
        group = pauli_commute[i]
        for Pauli_tuple in group:
            coeff = Pauli_tuple[1]
            final_expect_val += coeff * evaluation(per_miti_results[idx][i], shots = 1, Pauli = Pauli_tuple[0])  
    total += final_expect_val
    print("expectation value for mitigating qubit{}:{}".format(idx, final_expect_val) )

expectation value for mitigating qubit0:(-3.3359091103197245+0j)
expectation value for mitigating qubit1:(-3.3644912144954064+0j)
expectation value for mitigating qubit2:(-3.3516225532492903+0j)
expectation value for mitigating qubit3:(-3.3131583890829823+0j)
expectation value for mitigating qubit4:(-3.2514631009752333+0j)
expectation value for mitigating qubit5:(-3.3198281138693635+0j)


In [31]:
total/6

(-3.322745413665334+0j)

In [32]:
#untimigated result:
unmiti_expect_val = 0
for i in range(0, len(pauli_commute)):
    group = pauli_commute[i]
    for Pauli_tuple in group:
        coeff = Pauli_tuple[1]
        unmiti_expect_val += coeff * evaluation(unmiti_list[i], shots = total_counts(unmiti_list[i]), Pauli = Pauli_tuple[0])

In [33]:
unmiti_expect_val

(-3.3380599474310233+0j)